In [ ]:
import torch, math
import torch.optim as optim
x = torch.tensor([4.5], requires_grad = True)
#y = x**2
#print(x,y)
optimizer = optim.SGD([x],lr = 0.1)#,lr=0.01, momentum=0.9)
for i in range(20):
    optimizer.zero_grad()
    y = torch.square(x)
    y.backward()
    print(x,y)
    optimizer.step()    

In [ ]:
# initial coord
xy = torch.randn(2,requires_grad=True)*10
# grid 
x_coord = torch.linspace(-5.12,5.12,100)#.reshape(-1,1)
y_coord = torch.linspace(-5.12,5.12,100)#.reshape(-1,1)
x_mesh,y_mesh = torch.meshgrid(x_coord, y_coord, indexing='ij')
xy_mesh_stack = torch.stack((x_mesh,y_mesh))
print(xy_mesh_stack.shape)
# target function to find optimum/minimum
A,n,PI = 10,2,torch.tensor(math.pi)
f = lambda coord: A*n+torch.sum(torch.square(coord)-A*torch.cos(2*PI*coord),dim=0)

print(f(xy_mesh_stack).shape)
print(torch.min(f(xy_mesh_stack)))

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np

fig, ax = plt.subplots(subplot_kw={"projection": "3d"},figsize=(10,10))
surf = ax.plot_surface(x_mesh, y_mesh, f(xy_mesh_stack), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
plt.show()

```
output
torch.Size([2, 100, 100])
torch.Size([100, 100])
tensor(1.0520)
```
![3d view of 2d rastrigin](rast/rastrigin2d.png)
 

In [ ]:
deltaX = (max(x_coord) - min(x_coord))/10
deltaY = (max(y_coord) - min(y_coord))/10
xmin = min(x_coord) - deltaX
xmax = max(x_coord) + deltaX
ymin = min(y_coord) - deltaY
ymax = max(y_coord) + deltaY

fig = plt.figure(figsize=(8,8))
ax = fig.gca()
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
cfset = ax.contourf(x_mesh, y_mesh, f(xy_mesh_stack), cmap='coolwarm')
#ax.imshow(np.rot90(f), cmap='coolwarm', extent=[xmin, xmax, ymin, ymax])
#cset = ax.contour(x_mesh, y_mesh, f(xy_mesh_stack), colors='k')
#ax.clabel(cset, inline=1, fontsize=10)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.title('2D plot')

![bird view of 2d rastrigin](rast/rastrigin2dbird.png)

In [ ]:
import torch
x = torch.rand(2,requires_grad=True)*10
print(x)

# Sampling Rastrigin, unbounded random walk, bounded unif prior.

In [ ]:
from mcmc import MetropolisHasting, RastriginRandomWalkUnifPrior

for i in range(2):
    mhsampler = MetropolisHasting(10000,RastriginRandomWalkUnifPrior)
    mhsampler.performSample()
    print(mhsampler.target._last_sample_index)

    X = mhsampler.target._samples_store
    import matplotlib.pyplot as plt
    #plt.scatter(X[:, 0], X[:, 1])#, s=50, c = truth)
    #plt.title(f"Rastrigin sampling")
    #plt.xlabel("x")
    #plt.ylabel("y")
    #plt.show()

    x,y = X[:,0].numpy(), X[:,1].numpy()
    h =plt.hist2d(x, y,bins = 100)
    plt.colorbar(h[3])
    plt.show()
    bp = -1

![](rast/rastrigin_unbound_rw.png)

# RastriginRandomWalk Normal Prior 

## THIS ONE IS ACTUALLY BROKEN AND OF ALL THE SAME PROPOSAL VARIANCE  = 0.1

In [ ]:
from mcmc import MetropolisHasting, RastriginRandomWalkNormPrior

import matplotlib.pyplot as plt

fig, axes = plt.subplots(2,3,figsize = (12, 12))
PROPOSAL_STDS = [0.1,0.3,0.5]

for j,std in enumerate(PROPOSAL_STDS):
    for i in range(2):
        total_samples = 20000
        distr = RastriginRandomWalkNormPrior(total_samples,proposal_std=std,prior_std=2)
        mhsampler = MetropolisHasting(distr)
        mhsampler.performSample()
        #print(mhsampler.target._last_sample_index)

        X = mhsampler.target._samples_store

        x,y = X[:,0].numpy(), X[:,1].numpy()
        axes[i][j].hist2d(x, y,bins = 100)
    #plt.colorbar(axes[i][3])
#plt.show()
bp = -1

![](rast/rastrigin_rw_norm.png)

In [ ]:
from mcmc import MetropolisHasting, RastriginTruncNorm

import matplotlib.pyplot as plt

fig, axes = plt.subplots(2,3,figsize = (8, 8))
PROPOSAL_STDS = [0.05,0.1,0.5]

for j,std in enumerate(PROPOSAL_STDS):
    for i in range(2):
        total_samples = 20000
        distr = RastriginTruncNorm(total_samples,proposal_std=std,prior_std=2)
        mhsampler = MetropolisHasting(distr)
        mhsampler.performSample()
        #print(mhsampler.target._last_sample_index)

        X = mhsampler.target._samples_store

        x,y = X[:,0].numpy(), X[:,1].numpy()
        axes[i][j].hist2d(x, y,bins = 100)
    #plt.colorbar(axes[i][3])
#plt.show()
bp = -1

![](rast/rastrigin_trunc_norm.png)

In [9]:
from scipy.stats import truncnorm
#truncnorm.rvs(-5,5,loc=10,scale = 2)
my_std = 2
my_mean = 10
myclip_a = -10
myclip_b = -5

a, b = (myclip_a - my_mean) / my_std, (myclip_b - my_mean) / my_std
truncnorm.rvs(a,b,loc = my_mean, scale= my_std,size = 10)

array([-5.01917769, -5.19591141, -5.20441466, -5.19770292, -5.03653304,
       -5.3529195 , -5.27467897, -5.06163705, -5.18629912, -5.3863685 ])